In [70]:
import copy
from queue import PriorityQueue
import itertools

class EightPuzzle:
    hash = {}

    goal = [
        [1,2,3],
        [4,5,6],
        [7,8,0],
    ]

    #converts gameboard to hashable string
    @staticmethod
    def convert(gameboard: list[list[int]]):
        flat_gameboard = list(itertools.chain.from_iterable(gameboard))
        string = ""
        for i in range(len(flat_gameboard)):
            string += str(flat_gameboard[i])
        return string

    @staticmethod
    def empty_square(gameboard: list[list[int]]):
        for i in range(len(gameboard)):
            for j in range(len(gameboard[0])):
                if gameboard[i][j] == 0:
                    return (i,j)
        print("no empty square found")
        return (-1,-1)

    @staticmethod
    def operators(gameboard: list[list[int]]):
        (i,j) = EightPuzzle.empty_square(gameboard)
        if i < 0 or j < 0:
            return []

        operators = []

        #move down
        if i < len(gameboard)-1:
            tempboard = copy.deepcopy(gameboard)
            tempboard[i][j], tempboard[i+1][j] = tempboard[i+1][j], tempboard[i][j]
            operators.append(tempboard)
        #move up
        if i > 0:
            tempboard = copy.deepcopy(gameboard)
            tempboard[i][j], tempboard[i-1][j] = tempboard[i-1][j], tempboard[i][j]
            operators.append(tempboard)
        #move right
        if j < len(gameboard[0])-1:
            tempboard = copy.deepcopy(gameboard)
            tempboard[i][j], tempboard[i][j+1] = tempboard[i][j+1], tempboard[i][j]
            operators.append(tempboard)
        #move left
        if j > 0:
            tempboard = copy.deepcopy(gameboard)
            tempboard[i][j], tempboard[i][j-1] = tempboard[i][j-1], tempboard[i][j]
            operators.append(tempboard)
        
        return operators

    @staticmethod
    def is_goal(gameboard) -> bool:
        return gameboard == EightPuzzle.goal

    @staticmethod
    def print_gameboard(gameboard: list[list[int]]):
        print("[")
        for row in gameboard:
            print("\t", end="")
            for item in row:
                print(item, end=", ")
            print()
        print("]")



    @staticmethod
    def reset_hash():
        hash = {}

examples = [
    [
        [1,2,3],
        [4,5,6],
        [7,8,0],
    ],
    [
        [1,2,3],
        [4,5,6],
        [0,7,8],
    ],
    [
        [1,2,3],
        [5,0,6],
        [4,7,8],
    ],
    [
        [1,3,6],
        [5,0,2],
        [4,7,8],
    ],
    [
        [1,3,6],
        [5,0,7],
        [4,8,2],
    ],
    [
        [1,6,7],
        [5,0,3],
        [4,8,2],
    ],
    [
        [7,1,2],
        [4,8,5],
        [6,3,0],
    ],
    [
        [0,7,2],
        [4,6,1],
        [3,5,8],
    ]
]
print("Example 1:")
EightPuzzle.print_gameboard(examples[0])
print("is goal: ", EightPuzzle.is_goal(examples[0]))
print("convertion to string: ", EightPuzzle.convert(examples[0]))
print("Operators:")
for operator in EightPuzzle.operators(examples[0]):
    EightPuzzle.print_gameboard(operator)

print("---------------------------------------------------------------------")

print("Example 2:")
EightPuzzle.print_gameboard(examples[1])
print("is goal: ", EightPuzzle.is_goal(examples[1]))
print("convertion to string: ", EightPuzzle.convert(examples[1]))
print("Operators:")
for operator in EightPuzzle.operators(examples[1]):
    EightPuzzle.print_gameboard(operator)

Example 1:
[
	1, 2, 3, 
	4, 5, 6, 
	7, 8, 0, 
]
is goal:  True
convertion to string:  123456780
Operators:
[
	1, 2, 3, 
	4, 5, 0, 
	7, 8, 6, 
]
[
	1, 2, 3, 
	4, 5, 6, 
	7, 0, 8, 
]
---------------------------------------------------------------------
Example 2:
[
	1, 2, 3, 
	4, 5, 6, 
	0, 7, 8, 
]
is goal:  False
convertion to string:  123456078
Operators:
[
	1, 2, 3, 
	0, 5, 6, 
	4, 7, 8, 
]
[
	1, 2, 3, 
	4, 5, 6, 
	7, 0, 8, 
]


In [71]:
#Node for Eight Puzzle
class Node:
    def __init__(self, gameboard, value, depth) -> None:
        self.gameboard = gameboard
        self.value = value 
        self.depth = depth
    def __lt__(self, other):
        return self.value < other.value

    def is_goal(self):
        return EightPuzzle.is_goal(self.gameboard)
    def expand(self, q: PriorityQueue, qfunc):
        qfunc(q, self, EightPuzzle.operators(self.gameboard))


def uniform_cost(q: PriorityQueue, curr_node: Node, operators: list[list[list[int]]]):
    for operator in operators:
        if EightPuzzle.convert(operator) not in EightPuzzle.hash:
            q.put(Node(operator, curr_node.value+1, curr_node.depth+1))
            EightPuzzle.hash[EightPuzzle.convert(operator)] = True

def misplaced_tile(operator: list[list[int]]):
    num_tiles = 0
    for i in range(len(operator)):
        for j in range(len(operator[0])):
            if operator[i][j] != EightPuzzle.goal[i][j]:
                num_tiles += 1
    return num_tiles

def a_star(q: PriorityQueue, heuristic, curr_node: Node, operators: list[list[list[int]]]):
    for operator in operators:
        if EightPuzzle.convert(operator) not in EightPuzzle.hash:
            q.put(Node(operator, heuristic(operator) + curr_node.depth+1, curr_node.depth+1))
            EightPuzzle.hash[EightPuzzle.convert(operator)] = True

def a_star_misplaced_tile(q: PriorityQueue, curr_node: Node, operators: list[list[list[int]]]):
    a_star(q, misplaced_tile, curr_node, operators)

def search(initial_state, qfunc):
    pq = PriorityQueue()
    pq.put(Node(initial_state, 0, 0))
    max_depth = -1
    num_expanded_nodes = 0
    while True:
        if pq.empty():
            print("failure")
            return
        node = pq.get()

        if node.depth > max_depth:
            max_depth = node.depth
            print("max_depth so far: ", max_depth)

        if node.is_goal():
            # print stuff here
            print('found goal state at depth: ', node.value)
            print('Number of expanded nodes: ', num_expanded_nodes)
            print(node.gameboard)
            return node
        node.expand(pq, qfunc)
        num_expanded_nodes += 1

EightPuzzle.reset_hash()
search(examples[2], a_star_misplaced_tile)

max_depth so far:  0
max_depth so far:  1
max_depth so far:  2
max_depth so far:  3
max_depth so far:  4
found goal state at depth:  4
[[1, 2, 3], [4, 5, 6], [7, 8, 0]]
